In [1]:
from deap import base, creator, tools, algorithms
import numpy as np
import pandas as pd
import random
import plotly.express as px

In [2]:
# Load data
MAX_WEIGHT = 6404180

weights = profits = []

with open('p08_w.txt') as w_file:
  weights = w_file.readlines()

with open('p08_p.txt') as p_file:
  profits = p_file.readlines()

weights = [int(x.strip()) for x in weights]
profits = [int(x.strip()) for x in profits]

data = pd.DataFrame({'weight': weights, 'profit': profits})
count = data.count()
assert count[0] == count[1]
N = data.count()[0]
print(f'There are {N} items to choose')
print(f'Max weight is {MAX_WEIGHT}')
data.head()

There are 24 items to choose
Max weight is 6404180


,weight,profit
0,382745,825594
1,799601,1677009
2,909247,1676628
3,729069,1523970
4,467902,943972


In [3]:
# Define evaluation function. 
# In case that the sum of the weights is greater than the maximum weight, profit is zero.
[0,1,0,0,0,1]
def func_eval(ind, ):
  """Evaluation function for the individual."""
  return data.profit[ind].sum(),

def feasible(ind, ):
  """Feasibility function for the individual. Returns True if feasible False
  otherwise."""
  weight = data.weight[ind].sum()
  return weight <= MAX_WEIGHT

In [4]:
toolbox = base.Toolbox()

toolbox.register('select', tools.selRoulette)
toolbox.register('mate', tools.cxOnePoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.1)
toolbox.register('evaluate', func_eval)
toolbox.decorate('evaluate', tools.DeltaPenalty(feasible, 0.0))

creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox.register('bit', random.choice, seq=[True, False])
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.bit, n=N)


toolbox.register('population', tools.initRepeat, list, toolbox.individual)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register('min', np.min)
stats.register('max', np.max)
stats.register('mean', np.mean)
stats.register('std', np.std)
stats.register('median', np.median)

pop = toolbox.population(n=20)

hof1 = tools.HallOfFame(3)
hof2 = tools.HallOfFame(3)
hof3 = tools.HallOfFame(3)


In [5]:
from concurrent.futures import ThreadPoolExecutor

n_experiments = 10
n_gens = 1000

def eaSimple(_):
  _, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_gens, stats=stats, halloffame=hof1, verbose=False)
  log = pd.DataFrame(log)
  return log

def eaMuPlusLambda(_):
  _, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=3, lambda_=20, cxpb=0.7, mutpb=0.2, ngen=n_gens, stats=stats, halloffame=hof2, verbose=False)
  log = pd.DataFrame(log)
  return log

def eaMuCommaLambda(_):
  _, log = algorithms.eaMuCommaLambda(pop, toolbox, mu=3, lambda_=20, cxpb=0.7, mutpb=0.2, ngen=n_gens, stats=stats, halloffame=hof3, verbose=False)
  log = pd.DataFrame(log)
  return log

def experiment(algorithm, n_experiments):
  with ThreadPoolExecutor() as executor:
    logs = executor.map(algorithm,range(n_experiments))
  return list(logs)


In [6]:
def get_summary(logs):
  summary = pd.DataFrame()
  maxs = [log['max'] for log in logs]
  assert len(maxs) == n_experiments
  summary['std'] = np.std(maxs, axis=0)
  summary['avg best'] = np.mean(maxs, axis=0)
  summary['avg+std best'] = summary['avg best'] + summary['std']
  summary['avg-std best'] = summary['avg best'] - summary['std'] 
  return summary

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def print_hof(hof):
  for ind in hof:
    print(f'profit: {func_eval(ind)[0]}')
    ind = np.array(ind)
    ind = np.column_stack((ind,np.zeros(N),))


In [8]:
def plot_bests(logs):
  summary = get_summary(logs)
  fig = px.line(summary, y=['avg+std best', 'avg-std best'],line_dash_sequence=['dot'])
  fig.add_trace(
    go.Scatter(
      x=summary.index, y=summary['avg best'],
      line_color='rgb(0,176,246)',
      name='avg best'))
  fig.show()


In [9]:
logs1 = experiment(eaSimple, n_experiments)

In [10]:
plot_bests(logs1)
print_hof(hof1)

profit: 13449995
profit: 13405920
profit: 13404773


In [11]:
logs2 = experiment(eaMuPlusLambda, n_experiments)

In [12]:
print_hof(hof2)
plot_bests(logs2)

profit: 13368292
profit: 13356038
profit: 13348682
profit: 13338536


In [13]:
logs3 = experiment(eaMuCommaLambda, n_experiments)

In [14]:
print_hof(hof3)
plot_bests(logs3)

profit: 13426914
profit: 13381258
profit: 13375684
profit: 13368037
